In [4]:
import pymongo
from pymongo import MongoClient
from pymongo import GEO2D
from datetime import datetime
from pprint import pprint
from bson.objectid import ObjectId
import re
import string
from geojson import Point

In [5]:
client = MongoClient('mongodb://localhost:27017')
db = client.cabbies
db.collection_names()

['sy_apr-14',
 'y_apr-14',
 'old_greenapr14',
 'old_yellowapr14',
 'sy2_apr-14',
 'uberapr14']

In [6]:
collection = db['y_apr-14']

In [7]:
def get_amount_of_messages(collection):
    return collection.count()  
get_amount_of_messages(collection)

14618759

In [8]:
collection.find_one()

{'_id': ObjectId('5a3a7ce2f8d78e92a94d5133'),
 'dropoff_Point': {'coordinates': [40.750822, -73.972513], 'type': 'Point'},
 'dropoff_datetime': '2014-04-08 12:14:30',
 'passenger_count': '1',
 'payment_type': 'CRD',
 'pickup_Point': {'coordinates': [40.757125, -73.975848], 'type': 'Point'},
 'pickup_datetime': '2014-04-08 12:04:09',
 'total_amount': '10.2',
 'trip_distance': '1.0'}

In [195]:
def convert_to_Point(collection):
    bulk = collection.initialize_unordered_bulk_op()
    counter = 0
    count2 = 0
    requests = []
    cursor = collection.find(modifiers={"$snapshot": True})
    for record in cursor:
        if 'dropoff_Point' in record:
            count2+=1
            continue
        droplat = float(record['dropoff_latitude'])
        droplong = float(record['dropoff_longitude'])
        picklat = float(record['pickup_latitude'])
        picklong = float(record['pickup_longitude'])
        dropPoint = Point((droplong,droplat))
        pickPoint = Point((picklong,picklat))    
        bulk.find({ '_id': record['_id']}).update({'$unset':{'dropoff_latitude':'',\
                               'dropoff_longitude':'',\
                               'pickup_latitude':'',\
                               'pickup_longitude':''},\
                     '$set':{'dropoff_Point' : dropPoint, 'pickup_Point' : pickPoint}})
        counter+=1
        if counter % 1000 == 0:
            pprint(counter)
            bulk.execute()
            bulk = collection.initialize_unordered_bulk_op()
    if (counter % 1000 != 0):
        bulk.execute()
    pprint(counter)
    pprint(count2)
    return
collection = db['sy_apr-14']
convert_to_Point(collection)
collection.find_one()

{'_id': ObjectId('5a3ea9c5f8d78e92a92d0ab2'),
 'dropoff_Point': {'coordinates': [40.750822, -73.972513], 'type': 'Point'},
 'dropoff_datetime': '2014-04-08 12:14:30',
 'passenger_count': '1',
 'payment_type': 'CRD',
 'pickup_Point': {'coordinates': [40.757125, -73.975848], 'type': 'Point'},
 'pickup_datetime': '2014-04-08 12:04:09',
 'total_amount': '10.2',
 'trip_distance': '1.0'}

In [ ]:
def switch_long_lat(collection):
    bulk = collection.initialize_unordered_bulk_op()
    counter = 0
    count2 = 0
    requests = []
    cursor = collection.find(modifiers={"$snapshot": True})
    for record in cursor:
        droplat = record['dropoff_Point']['coordinates'][0]
        droplong = record['dropoff_Point']['coordinates'][1]
        picklat = record['pickup_Point']['coordinates'][0]
        picklong = record['pickup_Point']['coordinates'][1]
        dropPoint = Point((droplong,droplat))
        pickPoint = Point((picklong,picklat))    
        bulk.find({ '_id': record['_id']}).update({'$set':{'dropoff_Point' : dropPoint, 'pickup_Point' : pickPoint}})
        counter+=1
        if counter % 1000 == 0:
            pprint(counter)
            bulk.execute()
            bulk = collection.initialize_unordered_bulk_op()
    if (counter % 1000 != 0):
        bulk.execute()
    pprint(counter)
    pprint(count2)
    return

In [207]:
#from string to datetime, float, int
def convert_from_string(date_format, collection):
    bulk = collection.initialize_unordered_bulk_op()
    counter = 0
    count2 = 0
    requests = []
    cursor = collection.find(modifiers={"$snapshot": True})
    for record in cursor:
        #if type(record['passenger_count']) is int:
        #    count2+=1
        #    continue
        passenger_count = int(record['passenger_count'])
        trip_distance = float(record['trip_distance'])
        total_amount = float(record['total_amount'])
        pickup_datetime = datetime.strptime(record['pickup_datetime'],date_format)
        pickup_weekday = pickup_datetime.weekday()
        dropoff_datetime = datetime.strptime(record['dropoff_datetime'],date_format)
        dropoff_weekday = dropoff_datetime.weekday()
        bulk.find({'_id':record['_id']}).update({'$set':{'passenger_count' : passenger_count,\
                                       'trip_distance' : trip_distance,\
                                       'total_amount' : total_amount,\
                                       'pickup_datetime' : pickup_datetime,\
                                       'pickup_weekday' : pickup_weekday,\
                                       'dropoff_datetime' : dropoff_datetime,\
                                       'dropoff_weekday' : dropoff_weekday}})
        counter+=1
        if counter % 1000 == 0:
            pprint(counter)
            bulk.execute()
            bulk = collection.initialize_unordered_bulk_op()
    if (counter % 1000 != 0):
        bulk.execute()
    pprint(counter)
    pprint(count2)
    return
date_format = '%Y-%m-%d %H:%M:%S'
collection = db['sy_apr-14']
convert_from_string(date_format, collection)
collection.find_one()

{'_id': ObjectId('5a3ea9c5f8d78e92a92d0ab2'),
 'dropoff_Point': {'coordinates': [40.750822, -73.972513], 'type': 'Point'},
 'dropoff_datetime': datetime.datetime(2014, 4, 8, 12, 14, 30),
 'dropoff_weekday': 1,
 'passenger_count': 1,
 'payment_type': 'CRD',
 'pickup_Point': {'coordinates': [40.757125, -73.975848], 'type': 'Point'},
 'pickup_datetime': datetime.datetime(2014, 4, 8, 12, 4, 9),
 'pickup_weekday': 1,
 'total_amount': 10.2,
 'trip_distance': 1.0}

In [204]:
#from string to datetime, float, int
def convert_to_string(date_format, collection):
    cursor = collection.find()
    for record in cursor:
        passenger_count = str(record['passenger_count'])
        trip_distance = str(record['trip_distance'])
        total_amount = str(record['total_amount'])
        pickup_datetime = datetime.strftime(record['pickup_datetime'],date_format)
        dropoff_datetime = datetime.strftime(record['dropoff_datetime'],date_format)
        collection.update_one({'_id':record['_id']},\
                              {'$set':{'passenger_count' : passenger_count,\
                                       'trip_distance' : trip_distance,\
                                       'total_amount' : total_amount,\
                                       'pickup_datetime' : pickup_datetime,\
                                       'dropoff_datetime' : dropoff_datetime}})
    return
date_format = '%Y-%m-%d %H:%M:%S'
collection = db['sy_apr-14']
convert_to_string(date_format, collection)
collection.find_one()

{'_id': ObjectId('5a3ea9c5f8d78e92a92d0ab2'),
 'dropoff_Point': {'coordinates': [40.750822, -73.972513], 'type': 'Point'},
 'dropoff_datetime': '2014-04-08 12:14:30',
 'passenger_count': '1',
 'payment_type': 'CRD',
 'pickup_Point': {'coordinates': [40.757125, -73.975848], 'type': 'Point'},
 'pickup_datetime': '2014-04-08 12:04:09',
 'total_amount': '10.2',
 'trip_distance': '1.0'}

In [150]:
#Checks how many headers has Nulls
def check_Nulls(collection):
    document = collection.find_one()
    print('total\t',collection.count())
    for header in document:
        filter = {header: None}
        print(header,'\t',collection.count(filter))
    return
collection = db['sy_apr-14']
check_Nulls(collection)



total	 50
_id 	 0
pickup_datetime 	 0
dropoff_datetime 	 0
passenger_count 	 0
trip_distance 	 0
pickup_longitude 	 0
pickup_latitude 	 0
dropoff_longitude 	 0
dropoff_latitude 	 0
payment_type 	 0
total_amount 	 0


In [151]:
#Checks how many headers has String Type
def check_Strings(collection):
    print('total\t',collection.count())
    document = collection.find_one()
    for header in document:
        filter = {header: {'$type': 2}}
        print(header,'\t',collection.count(filter))
    return
collection = db['sy_apr-14']
check_Strings(collection)


total	 50
_id 	 0
pickup_datetime 	 0
dropoff_datetime 	 0
passenger_count 	 0
trip_distance 	 0
pickup_longitude 	 50
pickup_latitude 	 50
dropoff_longitude 	 50
dropoff_latitude 	 50
payment_type 	 50
total_amount 	 0
